# Tensorflow with GPU

This notebook provides an introduction to computing on a [GPU](https://cloud.google.com/gpu) in Colab. In this notebook you will connect to a GPU, and then run some basic TensorFlow operations on both the CPU and a GPU, observing the speedup provided by using the GPU.


In [1]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

C:\ProgramData\Anaconda3\python.exe
3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
sys.version_info(major=3, minor=6, micro=5, releaselevel='final', serial=0)


In [1]:
import numpy as np
import pandas as pd

## Data Preprocessing.

In [13]:
ann=pd.read_csv('/Churn_Modelling_ANN.csv')
ann

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [14]:
X=ann.iloc[:,3:-1].values
y=ann.iloc[:,-1].values

In [15]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [16]:
print(y)

[1 0 1 ... 1 1 0]


In [17]:
ann

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


### Label Encoding.

In [19]:
from sklearn.preprocessing import LabelEncoder
enc=LabelEncoder()

In [20]:
ann.Geography=enc.fit_transform(ann.Geography)
ann.Gender=enc.fit_transform(ann.Gender)
ann.Surname=enc.fit_transform(ann.Surname)

In [21]:
ann

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,1115,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,1177,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,2040,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,289,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,1822,850,2,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,1999,771,0,1,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,1336,516,0,1,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,1570,709,0,0,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,2345,772,1,1,42,3,75075.31,2,1,0,92888.52,1


In [22]:
# One Hot Encoding for 'Geography'

In [23]:
ann.Geography

0       0
1       2
2       0
3       0
4       2
       ..
9995    0
9996    0
9997    0
9998    1
9999    0
Name: Geography, Length: 10000, dtype: int64

In [24]:
ann.Geography.value_counts()

0    5014
1    2509
2    2477
Name: Geography, dtype: int64

In [25]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [26]:
#col_trans=ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[0])],remainder='passthrough')
#X=np.array(col_trans.fit_transform(X))

on_hot=OneHotEncoder()
#ann.Geography=on_hot.fit_transform(ann.Geography)

### Splitting Dataset into Training and Testing.

- In order to train your model on your data, convert it to a numpy array first with X_train.values and y_train.values since a Keras model accepts as input a numpy array and not a pandas dataset

In [28]:
X=ann.iloc[:,3:-1]
y=ann.iloc[:,-1]

In [29]:
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,0,0,42,2,0.00,1,1,1,101348.88
1,608,2,0,41,1,83807.86,1,0,1,112542.58
2,502,0,0,42,8,159660.80,3,1,0,113931.57
3,699,0,0,39,1,0.00,2,0,0,93826.63
4,850,2,0,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,0,1,39,5,0.00,2,1,0,96270.64
9996,516,0,1,35,10,57369.61,1,1,1,101699.77
9997,709,0,0,36,7,0.00,1,0,1,42085.58
9998,772,1,1,42,3,75075.31,2,1,0,92888.52


In [30]:
from sklearn.model_selection import train_test_split

In [31]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0,test_size=0.2)

# Building ANN.

In [33]:
# Feature Scaling.
## Feature scaling is absolutely compulsory for deep learning.

In [34]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

In [35]:
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

# Initializing the ANN

### Adding the input layer, the first hidden layer, second hidden layer and the Output Layer.


In [37]:
import tensorflow as tf

In [38]:
# Sequential class is from 'keras models' module from keras library, which represents a class which initializes ANN to work as sequence of layers.
ann_seq=tf.keras.models.Sequential()


# How to know how many neurons are required for ANN? Its based on experimentations and by Hyper Parameters.

ann_seq.add(tf.keras.layers.Dense(units=6, activation='relu'))

# Adding 1st Input layer and Hidden layer using 'Dense' class. 
# Units: Units represents number of hidden layers, and Activation function is 'Relu'.

In [39]:
# Adding 2nd Layer with same number of units and activation function.

ann_seq.add(tf.keras.layers.Dense(units=6, activation='relu'))

In [40]:
# Adding Output Layer.

ann_seq.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Units here will 1 because we are having binary outcome which can take the outcome 1 or 0, where 1 neuron is sufficient to predict the class.
# Activation function for(binary) the output class we need to have 'Sigmoid Function' only; instead of 'Relu' because it allows to get ultimate predictions and also probabilities that binary outcome is 1.
# Activation function for non-binary is 'Softmax'

## Compiling the ANN.

In [41]:
# Compilation of ANN. 
# In complier we have to enter 3 parameters i.e 1.Optimizer,  2.Loss Function and 3.Metrics with S Parameter

# Adam Optimizer to get Stochastic Optimizer, Optimizer updates the weights through 'Stochastic Gradient Descent'.
# Loss function: For Binary classification it should be 'binary_ crossentropy' and for non-binary it should be 'categorical_crossentropy'
# Metrics: 'Accuracy' 

In [42]:

ann_seq.compile(optimizer='adam' ,loss='binary_crossentropy', metrics=['accuracy'])

## Training the ANN.

In [43]:

# Method is fit method and including 'batch_size' and 'Epochs'
# Batch_size: NO of predictions we required to have in a batch when compared to same number of real results. 32 is default value
# Epochs :The number of epochs is a hyperparameter that defines the number times that the learning algorithm will work through the entire training dataset. 
# One epoch means that each sample in the training dataset has had an opportunity to update the internal model parameters. An epoch is comprised of one or more batches.


In [44]:
ann_seq.fit(X_train,y_train,batch_size= 32, epochs=100 )

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5834 - accuracy: 0.7135
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4585 - accuracy: 0.8008
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4475 - accuracy: 0.7935
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4378 - accuracy: 0.8030
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4247 - accuracy: 0.8130
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4257 - accuracy: 0.8055
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4289 - accuracy: 0.8013
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4116 - accuracy: 0.8123
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4022 - accuracy: 0.8204
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3816 - accura

## To predict by giving inputs:


In [1]:

## Geography:France, 
## Credit Score:600, 
## Gender: Male, 
## Age:40, 
## Tenure:3 years, 
## Balance:$60000, 
## Number of Products:2, 
## Does this customer have a credit card: Yes,
## Is this customer an Active Member: Yes,
## Estimated Salary: $50000
## So, Should we say goodbye to that customer?

# For Above values, To get predictions we have to enter scaled values.

print(ann_seq.predict(sc.transform([[1,600,1,40,3,6000,2,1,1,50000]])))

## Predicting the Test set Results.

In [56]:
y_predict=ann_seq.predict(X_test)

# Using Logistic Regression Condition below.
y_predict=(y_predict>0.5)


In [61]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm=confusion_matrix(y_test,y_predict)
cm

array([[1472,  123],
       [ 254,  151]])

In [62]:
print(cm)
print(accuracy_score(y_test,y_predict))

[[1472  123]
 [ 254  151]]
0.8115
